In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.gridspec as gridspec


# Load and parse the MATLAB .mat file
def load_mat_file(filename):
    """
    Load a MATLAB .mat file and convert nested MATLAB structures to Python dictionaries.

    Parameters:
    - filename: Path to the .mat file

    Returns:
    - A dictionary with the loaded data
    """
    def _mat_to_dict(obj):
        if isinstance(obj, np.ndarray):
            return [_mat_to_dict(o) for o in obj] if obj.dtype == object else obj
        elif hasattr(obj, '_fieldnames'):
            return {field: _mat_to_dict(getattr(obj, field)) for field in obj._fieldnames}
        else:
            return obj

    mat_data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return {key: _mat_to_dict(value) for key, value in mat_data.items() if not key.startswith('__')}

# Plot raster of spike times across all stimuli for a specific neuron
def plot_raster(parsed_data, neuron_index, stimlabels, line_width=2.5, tick_height=0.9):
    """
    Generate raster plots showing spike times for a specific neuron across all stimuli.

    Parameters:
    - parsed_data: Loaded and parsed data from the .mat file
    - neuron_index: Index of the neuron to plot
    - line_width: Width of spike lines in the raster plot
    - tick_height: Height of spike ticks in the raster plot
    """
    neuron = parsed_data['alldata'][neuron_index]
    colors = ['red', 'green', 'blue', 'black']

    plt.figure(figsize=(10, 8))
    trial_counter = 0

    for stim_idx, stimulus in enumerate(neuron['data']):
        color = colors[stim_idx]
        for trial in stimulus['sweep']:
            plt.vlines(trial['spikes'], trial_counter + 0.5 - tick_height/2, trial_counter + 0.5 + tick_height/2, color=color, linewidth=line_width)
            trial_counter += 1

    # Add legend-like text box in the top right corner
    #legend_text = "".join([f"{stimlabels[i]}\n" for i in range(len(neuron['data']))])
    #colors_list = [colors(i) for i in range(len(neuron['data']))]

    # Add color-coded text
    for i, stimlabel in enumerate(stimlabels):
        color = colors[i]
        plt.text(1.02, 0.95 - i * 0.05, stimlabel, transform=plt.gca().transAxes, color=color, fontsize=15, verticalalignment='top')

    plt.xlabel('Time (ms)')
    plt.ylabel('Trial')
    plt.title(f'Neuron {neuron_index} - Raster Plot Across All Stimuli')
    plt.tight_layout()
    plt.show()

# Convert spike times to activity matrix using histogram binning
def spike_times_to_activity_matrix(parsed_data, neuron_index, bin_size=1, max_time=1200):
    """
    Convert spike times into an activity matrix using histogram binning.

    Parameters:
    - parsed_data: Loaded and parsed data
    - neuron_index: Index of the neuron to analyze
    - bin_size: Size of time bins (in ms)
    - max_time: Maximum time duration to consider (in ms)

    Returns:
    - activity_matrix: Binary matrix representing spike activity per bin
    - trial_counts: Number of trials per stimulus
    """
    neuron_data = parsed_data['alldata'][neuron_index]['data']
    all_trials = [trial['spikes'] for stimulus in neuron_data for trial in stimulus['sweep']]

    num_trials = len(all_trials)
    num_bins = max_time // bin_size
    bin_edges = np.arange(0, max_time + bin_size, bin_size)

    activity_matrix = np.zeros((num_trials, num_bins))

    for trial_idx, trial in enumerate(all_trials):
        spike_times = np.array(trial, ndmin=1)
        activity_matrix[trial_idx], _ = np.histogram(spike_times, bins=bin_edges)

    return activity_matrix, [len(stimulus['sweep']) for stimulus in neuron_data]

# Perform PCA on activity matrix
def neural_coding_pca(data):
    """
    Perform Principal Component Analysis (PCA) on the activity matrix.

    Parameters:
    - data: 2D NumPy array (trials x time bins)

    Returns:
    - eigenvals: Eigenvalues of principal components
    - eigenvecs: Eigenvectors (principal components)
    - transformed_data: Data projected into principal component space
    """
    total_spikes = np.sum(data)
    num_trials = data.shape[0]
    norm_factor = total_spikes / num_trials if num_trials > 0 else 0

    if norm_factor == 0:
        return 0, 0, data  # No spikes present

    normalized_data = data / norm_factor
    centered_data = normalized_data - np.mean(normalized_data, axis=1, keepdims=True)

    covariance_matrix = np.cov(centered_data, rowvar=False)
    eigenvals, eigenvecs = np.linalg.eigh(covariance_matrix)

    sorted_indices = np.argsort(eigenvals)[::-1]
    eigenvals = eigenvals[sorted_indices]
    eigenvecs = eigenvecs[:, sorted_indices]

    transformed_data = np.dot(centered_data, eigenvecs)

    return eigenvals, eigenvecs, transformed_data

# Classify neural responses and visualize the PCA space
def classify_and_plot(transformed_data, trial_counts, num_pc_to_use, stimlabels):
    """
    Classify neural responses based on PCA-transformed data and visualize the results.

    Parameters:
    - transformed_data: A 2D array where each row is a trial and each column is a principal component.
    - trial_counts: A list with the number of trials for each stimulus.
    - num_pc_to_use: The number of principal components to use for the classification.
    """

    # Import ListedColormap to create our own color map later for plotting.
    from matplotlib.colors import ListedColormap

    # Create an array 'allstimuli' that contains the label for each trial.
    # For example, if trial_counts is [5, 7], then this will create an array:
    # [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
    allstimuli = np.concatenate([
        np.full(count, stim) for stim, count in enumerate(trial_counts)
    ])

    # Extract the first two principal components (columns) for 2D plotting.
    PC1 = transformed_data[:, 0]  # First principal component (x-axis)
    PC2 = transformed_data[:, 1]  # Second principal component (y-axis)

    # Define colors for each stimulus. Make sure the number of colors is at least the number of stimuli.
    custom_colors = ['red', 'green', 'blue', 'black']

    # Create a new figure with a specific size (width 8 inches, height 6 inches).
    plt.figure(figsize=(8, 6))

    # Create a scatter plot of the data points using PC1 and PC2.
    # Each point is colored based on its stimulus label.
    plt.scatter(PC1, PC2, c=[custom_colors[stim] for stim in allstimuli], edgecolor='k')

    # Label the x-axis and y-axis.
    plt.xlabel('PC1', fontsize=16)
    plt.ylabel('PC2', fontsize=16)
    plt.tick_params(labelsize=14)  # Set tick label font size

    # Create an instance of the Linear Discriminant Analysis (LDA) classifier.
    lda = LinearDiscriminantAnalysis()

    # Train the LDA classifier using the first 'num_pc_to_use' principal components.
    lda.fit(transformed_data[:, :num_pc_to_use], allstimuli)

    # Define the boundaries for a grid that will cover our plot area.
    # We add a margin of 1 unit to the minimum and maximum values.
    x_min, x_max = PC1.min() - 1, PC1.max() + 1
    y_min, y_max = PC2.min() - 1, PC2.max() + 1

    # Create a grid of points over the defined range (200 points along each axis).
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

    # Combine the grid points into a 2D array where each row is a point (x, y).
    grid_points = np.c_[xx.ravel(), yy.ravel()]

    # If we are using more than 2 principal components, we need to add extra dimensions.
    # Since we can only plot in 2D, we use the average value for the extra dimensions.
    if num_pc_to_use > 2:
        # Calculate the average values of the additional dimensions (from the 3rd to the selected one).
        extra_dims = np.mean(transformed_data[:, 2:num_pc_to_use], axis=0)
        # For each grid point, append these average values so that the input has the correct dimensions.
        grid_points = np.hstack([grid_points, np.tile(extra_dims, (grid_points.shape[0], 1))])

    # Predict the class (stimulus) for each point in the grid using the trained LDA.
    Z = lda.predict(grid_points[:, :num_pc_to_use])
    # Reshape the predictions to match the shape of the grid for plotting.
    Z = Z.reshape(xx.shape)

    # Create a custom colormap using our defined colors (only as many as there are stimuli).
    cmap = ListedColormap(custom_colors[:len(trial_counts)])

    # Plot the decision boundaries by coloring the grid according to predicted classes.
    # The 'alpha' value makes this layer semi-transparent.
    plt.contourf(xx, yy, Z, alpha=0.2, cmap=cmap)

    # Predict the classes for the original data points to calculate accuracy.
    predictions = lda.predict(transformed_data[:, :num_pc_to_use])
    accuracy = np.mean(predictions == allstimuli) * 100  # Convert to percentage.

    # Set the plot title to show the classification accuracy.
    plt.title(f'Classification Accuracy = {accuracy:.2f}%', fontsize=16)

    # Create a custom legend: for each stimulus, make a marker with the correct color.
    handles = [plt.Line2D([0], [0], marker='o', color='w',
                            markerfacecolor=custom_colors[i], markersize=10,
                            label=f'{stimlabels[i]}') for i in range(len(trial_counts))]
    plt.legend(handles=handles, title="Stimuli", fontsize=14)

    # Finally, display the plot.
    plt.show()


In [ ]:
# Load data from computer
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load data from google's copy
data_path = 'data_1ms_res.mat'
parsed_data = load_mat_file(data_path)

# Plot raster of a chosen neuron
# Plot for different neurons to visualize interesting responses
# each row is a trial, and different stimulus classes are color coded.
neuron_index = 2

stimlabels = [str(parsed_data['alldata'][neuron_index]['data'][stimi]['stim'][0]) \
                  for stimi in range(len(parsed_data['alldata'][neuron_index]['data']))]


plot_raster(parsed_data, neuron_index=neuron_index, stimlabels=stimlabels)


In [ ]:
# Convert spike times to a matrix for further processing.
# Below select bin size and see how that affects how data is represented.

neuron_index = 2
bin_size = 10
activity_matrix, trial_counts = spike_times_to_activity_matrix(parsed_data, neuron_index=neuron_index, bin_size=bin_size, max_time=1200)

# Plot the activity matrix using pcolormesh with grayscale
plt.figure(figsize=(25, 8))
plt.pcolormesh(activity_matrix, cmap='gray')

# Add horizontal dashed lines to indicate stimulus boundaries
cumulative_trials = np.cumsum(trial_counts)
for line in cumulative_trials[:-1]:  # Exclude the last cumulative value as it's the total
    plt.axhline(line, color='red', linestyle='--')

plt.xlabel('Time (ms)')
plt.ylabel('Trial')
plt.title('Activity Matrix for Neuron 2')
plt.show()

In [ ]:
# the code below performs PCA in the matrix calculated above.
# in the figure below you can see how the variance of the data is distributed along the principal components

eigenvals, eigenvecs, transformed_data = neural_coding_pca(activity_matrix)

total_variance = np.sum(np.abs(eigenvals))
explained_variance = (eigenvals / total_variance) * 100

plt.figure(figsize=(8, 6))
plt.plot(explained_variance, '-o')
plt.xlabel('Principal Component', fontsize=16)
plt.ylabel('% Explained Variance', fontsize=16)
plt.gca().tick_params(labelsize=14)
plt.show()

In [ ]:
# Visualize PC weights alongside their associated time bins
# See that the bins with higher variance will tend to be the ones with higher weights in the first PC

# Select the PC you want to look, 1st PC starts at 0
pc_i = 0

plt.figure(figsize=(20, 10))

# GridSpec layout for PC weights and activity matrix
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

nbins = len(eigenvecs[:, 0])

# Plot the user-defined PC
ax = plt.subplot(gs[0])
ax.plot(np.arange(nbins) + 0.5, eigenvecs[:, pc_i])
ax.set_xlim(0, len(eigenvecs[:, pc_i]))
ax.set_ylabel(f'PC{pc_i + 1}')

# Plot Activity Matrix
ax_activity = plt.subplot(gs[1])

# Aligning the pcolormesh with the PCs
extent = [0, len(eigenvecs[:, 0]), 0, activity_matrix.shape[0]]
im = ax_activity.imshow(activity_matrix, cmap='gray', aspect='auto', extent=extent)

# Add horizontal dashed lines to indicate stimulus boundaries
cumulative_trials = np.cumsum(trial_counts)
for line in cumulative_trials[:-1]:  # Exclude the last cumulative value as it's the total
    plt.axhline(line, color='red', linestyle='--')

plt.tight_layout()
plt.show()



In [ ]:
# Visualize the projection of the activity matrix onto the principal components
# This helps to observe how the trials are represented in the new PCA space

import matplotlib.gridspec as gridspec

num_pc_to_plot = 3  # Number of principal components to visualize

plt.figure(figsize=(20, 10))

# Plot the PCA-transformed data using pcolormesh
plt.pcolormesh(transformed_data[:, :num_pc_to_plot], cmap='Reds')

# Add horizontal dashed lines to indicate stimulus boundaries
cumulative_trials = np.cumsum(trial_counts)
for line in cumulative_trials[:-1]:  # Exclude the last cumulative value as it's the total
    plt.axhline(line, color='red', linestyle='--')

plt.tight_layout()
plt.show()

In [ ]:
num_pc_to_use = 2


x = transformed_data[:, :num_pc_to_use].copy()

classify_and_plot(x, trial_counts, num_pc_to_use, stimlabels=stimlabels)